In [1]:
import tenseal as ts
# Setup TenSEAL context   
# TenSEAL Context is a special object that holds different encryption keys and parameters for 
# us, so that we only need to use a single object to make our encrypted computation instead
# of managing all the keys and the HE details. Basically, we will want to create a single
# TenSEAL context before doing our encrypted vectors of either integers(using BFV) or real 
# numbers (using CKKS)/
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60,40,40, 60]
    
)

context.generate_galois_keys()
context.global_scale = 2**40

v1 = [0,1,2,3,4]
v2 = [4,3,2,1,0]

#encrypted vectors
enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)

result = enc_v1 + enc_v2
result.decrypt() # ~ [4,4,4,4,4]

result = enc_v1.dot(enc_v2)
result.decrypt() # ~ [10]

matrix = [
       [73, 0.5, 8],
        [81, -5, 66],
        [-100, -78, -2],
        [0, 9, 17],
        [69, 11, 10],   
]
result = enc_v1.matmul(matrix)
result.decrypt() # ~ [157, -90, 153]


[157.00002129249523, -90.00001212247784, 153.00002052326667]

In [2]:
import tenseal as ts

context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree= 4096, plain_modulus= 1032193)
context



In [3]:
public_context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
print("Is the context private?", ("Yes" if public_context.is_private() else "No"))
print("Is the context public?", ("Yes" if public_context.is_public() else "No"))

sk = public_context.secret_key()

# the context will drop the secret_key at this point
public_context.make_context_public() 
print("Secret-key dropped")
print("Is the context private?", ("Yes" if public_context.is_private() else "No"))
print("Is the context public?", ("Yes" if public_context.is_public() else "No"))




Is the context private? Yes
Is the context public? No
Secret-key dropped
Is the context private? No
Is the context public? Yes


In [4]:
plain_vector = [60, 66, 73, 81, 90]
encrypted_vector = ts.bfv_vector(context, plain_vector)
print("We just encrypted our plaintext vector of size : ", encrypted_vector.size())
encrypted_vector

We just encrypted our plaintext vector of size :  5


In [5]:
# Here we encrypted a vector of integers into a BFVVector, a vector type that uses the BFV
# scheme. Now we can do both addition, substraction and multiplication in an element-wise
# fashion with other encrypted or plain vectors
add_result = encrypted_vector + [1,2,3,4,5]
print(add_result.decrypt())


[61, 68, 76, 85, 95]


In [9]:
sub_result = encrypted_vector - [1,2,3,4,5]
print(sub_result.decrypt())

[59, 64, 70, 77, 85]


In [8]:
mul_result = encrypted_vector * [1,2,3,4,5]
print(mul_result.decrypt())

[60, 132, 219, 324, 450]


In [10]:
encrypted_add = add_result + sub_result
print(encrypted_add.decrypt())

[120, 132, 146, 162, 180]


In [11]:
encrypted_sub = encrypted_add - encrypted_vector
print(encrypted_sub.decrypt())

[60, 66, 73, 81, 90]


In [12]:
encrypted_mul = encrypted_add * encrypted_sub
print(encrypted_mul.decrypt())

[7200, 8712, 10658, 13122, 16200]


In [13]:
from time import time 

t_start = time()
_ = encrypted_add * encrypted_mul
t_end = time()
print("c2c multiply time: {} ms".format((t_end - t_start) * 1000))

t_start = time()
_ = encrypted_add * [1,2,3,4,5]
t_end = time()
print("c2p multiply time: {} ms".format((t_end - t_start) * 1000))

c2c multiply time: 3.6003589630126953 ms
c2p multiply time: 0.5462169647216797 ms


In [15]:
print("Automatic relinearization is:", ("on" if context.auto_relin else "off"))
print("Automatic rescaling is:", ("on" if context.auto_rescale else "off"))
print("Automatic modulus switching is:", ("on" if context.auto_mod_switch else "off"))

Automatic relinearization is: on
Automatic rescaling is: off
Automatic modulus switching is: off


In [16]:
# this should throw an error as the global_scale isn't defined yet
try:
    print("global_scale:", context.global_scale)
except ValueError:
    print("The global_scale isn't defined yet")
# you can define it to 2 ** 20 for instance
context.global_scale = 2 ** 20
print("global_scale:", context.global_scale)

The global_scale isn't defined yet
global_scale: 1048576.0
